# Handling Offline Video

Reference: https://www.youtube.com/watch?v=AxIc-vGaHQ0

---
### Mount to Google Drive
Mount to your Google Drive to access the images and model files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DataFolder = "/content/drive/MyDrive/SDAAI/data"

---
### Import necessary libraries

In [ ]:
import cv2
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import clear_output
from time import sleep
from tqdm import tqdm

---
A function to play the MP4 file using JS.

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [ ]:
show_local_mp4_video(DataFolder + "/LU06_videos/Demo1.mp4", width = 500)

# Retrieve basic info of the video

In [ ]:
# Load in video capture
cap = cv2.VideoCapture(DataFolder + "/LU06_videos/Demo1.mp4")

# Total number of frames in video
total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
print(f'Total number of frames : {total_frame}')

# Video height and width
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
print(f'Height : {height}, Width : {width}')

# Get frames per second
fps = cap.get(cv2.CAP_PROP_FPS)
print(f'FPS : {fps:0.2f}')

cap.release()

# Read in one frame

In [ ]:
cap = cv2.VideoCapture(DataFolder + "/LU06_videos/Demo1.mp4")
ret, img = cap.read()
print(f'Returned {ret} and img of shape {img.shape}')
cap.release()

# Display the frame

In [ ]:
img_display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_display)
plt.show()

# Display multiple frames at interval of 50 frames

In [ ]:
fig, axs = plt.subplots(4, 4, figsize=(20, 10))
axs = axs.flatten()

cap = cv2.VideoCapture(DataFolder + "/LU06_videos/Demo1.mp4")
total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
img_idx = 0
for frame in tqdm(range(total_frame)):
    ret, img = cap.read()
    if ret == False:
        break
    if frame % 50 == 0:
        axs[img_idx].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        axs[img_idx].set_title(f'Frame: {frame}')
        axs[img_idx].axis('off')
        img_idx += 1

    frame += 1

plt.tight_layout()
plt.show()
cap.release()

# Write to file

In [ ]:
VIDEO_CODEC = "mp4v"
fps = 59.94
width = 1280
height = 720
out = cv2.VideoWriter("out_test.mp4",
                cv2.VideoWriter_fourcc(*VIDEO_CODEC),
                fps,
                (width, height))

cap = cv2.VideoCapture(DataFolder + "/LU06_videos/Demo1.mp4")
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

for frame in tqdm(range(n_frames), total=n_frames):
    ret, img = cap.read()
    if ret == False:
        break

    out.write(img)

out.release()
cap.release()

---
Convert the mp4v to H264 format.

In [ ]:
video_mp4 = "out_final.mp4"
!ffmpeg -y -loglevel info -i out_test.mp4 -vf scale=640:480 $video_mp4

In [ ]:
show_local_mp4_video(video_mp4, width = 500)

Notice the removal of sound from the video file. Video analysis in Computer Vision focuses on the images, ignoring the sound component.